---
title: Preprocessing
description: Preprocessing the data for future use
---

In [2]:
import pandas as pd
from IPython.display import display, HTML

### Preprocessing match results

In [3]:
# Column names : 'League', 'Country', 'Season', 'Date', 'Home', 'HomeGoals', 'Away', 'AwayGoals'
match_results = pd.read_csv('data/extracted_match_results.csv', parse_dates=['Date'])

# Fix encoding issue : renaming 'Fu\303\237ball-Bundesliga' to 'Bundesliga'
match_results['League'] = match_results['League'].replace('Fu<U+00C3><U+009F>ball-Bundesliga', 'Bundesliga')

# Map country codes to country names
country_map = {'ENG': 'England', 'ITA': 'Italy', 'FRA': 'France', 'GER': 'Germany', 'ESP': 'Spain', 'POR': 'Portugal', 'SCO': 'Scotland', 'POL': 'Poland', 'GRE': 'Greece', 'TUR': 'Turkey', 'SUI': 'Switzerland', 'NED': 'Netherlands', 'BEL': 'Belgium', 'AUT': 'Austria'}
match_results['Country'] = match_results['Country'].map(country_map)

In [4]:
match_results.describe()

,Season,Date,HomeGoals,AwayGoals
count,16463.000000,16463,16362.000000,16362.000000
mean,2019.000486,2019-01-10 07:48:02.755269376,1.536487,1.213299
min,2015.000000,2014-08-08 00:00:00,0.000000,0.000000
25%,2017.000000,2016-10-29 00:00:00,1.000000,0.000000
50%,2019.000000,2019-01-11 00:00:00,1.000000,1.000000
75%,2021.000000,2021-03-21 00:00:00,2.000000,2.000000
max,2023.000000,2023-06-11 00:00:00,10.000000,9.000000
std,2.581816,NaN,1.310644,1.166416


In [5]:
#| label: match_results
match_results.head()

,League,Country,Season,Date,Home,HomeGoals,Away,AwayGoals
0,Premier League,England,2015,2014-08-16,Manchester Utd,1.0,Swansea City,2.0
1,Premier League,England,2015,2014-08-16,Stoke City,0.0,Aston Villa,1.0
2,Premier League,England,2015,2014-08-16,Leicester City,2.0,Everton,2.0
3,Premier League,England,2015,2014-08-16,QPR,0.0,Hull City,1.0
4,Premier League,England,2015,2014-08-16,West Ham,0.0,Tottenham,1.0


Étant donné que nous ne sommes pas intéressés par les résultats des équipes, mais plutôt par le résultat de chaque équipe, nous modifierons cette base de données en divisant les résultats du match en deux lignes distinctes, une pour chaque équipe. Cela nous permettra de calculer les statistiques pour chaque équipe séparément.

In [6]:
def return_result(goal1, goal2):
    if goal1 > goal2:
        return 'win'
    elif goal1 < goal2:
        return 'loss'
    else:
        return 'draw'
    
match_results['HomeResult'] = match_results.apply(lambda x: return_result(x['HomeGoals'], x['AwayGoals']), axis=1)
match_results['AwayResult'] = match_results.apply(lambda x: return_result(x['AwayGoals'], x['HomeGoals']), axis=1)

home_results = match_results[['League', 'Country', 'Date', 'Home', 'HomeGoals', 'HomeResult']]
home_results = home_results.rename(columns={'Home': 'Team', 'HomeGoals': 'Goals', 'HomeResult': 'Result'})
home_results['isHome'] = True

away_results = match_results[['League', 'Country', 'Away', 'Date', 'AwayGoals', 'AwayResult']]
away_results = away_results.rename(columns={'Away': 'Team', 'AwayGoals': 'Goals', 'AwayResult': 'Result'})
away_results['isHome'] = False

match_results = pd.concat([home_results, away_results], ignore_index=True)

### Preprocessing head coach

In [7]:
#| label: head_coach

# Column names : 'Team', 'League', 'Country', 'HeadCoach', 'Appointed', 'EndDate', 'Tenure', 'Matches', 'Wins', 'Draws', 'Losses'
head_coach = pd.read_csv('data/extracted_head_coach.csv', parse_dates=['Appointed', 'EndDate'])
head_coach.head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses
0,Chelsea FC,Premier League,England,Mauricio Pochettino,2023-07-01,NaT,284,42,21,9,12
1,Chelsea FC,Premier League,England,Graham Potter,2022-09-08,2023-04-02,206,31,12,8,11
2,Chelsea FC,Premier League,England,Thomas Tuchel,2021-01-26,2022-09-07,589,100,63,19,18
3,Chelsea FC,Premier League,England,Frank Lampard,2019-07-04,2021-01-25,571,84,44,15,25
4,Chelsea FC,Premier League,England,Maurizio Sarri,2018-07-14,2019-06-30,351,63,40,11,12


In [8]:
head_coach.describe()

,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses
count,3501,3437,3501.000000,3501.000000,3501.000000,3501.000000,3501.000000
mean,1982-03-27 02:38:21.285347008,1983-02-23 18:37:48.722723328,608.261640,51.560411,22.558983,12.973436,16.027992
min,1886-06-26 00:00:00,1893-08-01 00:00:00,-242.000000,0.000000,0.000000,0.000000,0.000000
25%,1961-07-01 00:00:00,1963-04-01 00:00:00,186.000000,10.000000,2.000000,2.000000,4.000000
50%,1986-11-06 00:00:00,1987-10-01 00:00:00,364.000000,29.000000,10.000000,7.000000,10.000000
75%,2004-09-27 00:00:00,2005-05-28 00:00:00,730.000000,68.000000,28.000000,17.000000,21.000000
max,2024-03-19 00:00:00,2024-06-30 00:00:00,14613.000000,1490.000000,895.000000,323.000000,272.000000
std,NaN,NaN,817.590044,75.888020,39.502648,18.724610,20.571695


We need to filter head coach that were not active between 2015 to 2023.

In [9]:
# Remove head coach that were appointed after 2023 seasons
head_coach = head_coach[head_coach['Appointed'] <= '2024-01-14']
# Keep head coach that were dimissed after 2015 or that are still active
head_coach = head_coach[(head_coach['EndDate'] >= '2015-01-01') | (head_coach['EndDate'].isna())]

:::{caution}
Head coach appointments records, extracted from TransferMarkt, contains data related to head coach in that specific club : tenure, number of matches played, number of matches won, etc. Those datapoint goes beyond our cut-off date of 2023 end season (2024-01-14).

One important thing is that those feature beyond cut-off date still relate to a head coach appointment we have in our records. This guarantees us that metrics such *number of club head coach managed* are properly reflected and still relate to head coach performance.

However, this create an asymetry in our data, as certain data point are limited by a time-frame and others not.
Moreover, we must be careful in how we compare these datapoint to others dataset such as match results as it could easily bias our statistical study.

The only way we combine this dataset to match result is by extracting head coach tenure on day of a match. This does not bias our statistical study as it is a feature that is properly reflected by our cut-off date.
:::

:::{note}
I have considered computing Head Coach performance metric from match results but we would lose information on prior records as well as creating imbalance data for plot such as linear regression of head coach performance over head coach tenure : a long standing coach which would not been dismissed soon after our start date would have a lower number of matches, thus a performance metric with higher variance that would bias linear regression due to long tenure.
:::

In [10]:
# Ensuring there is only 1 head coach at a time in any given team.

head_coach_bis = head_coach.copy()
# Sort data by 'Team' and 'Appointed'
head_coach_bis = head_coach_bis.sort_values(['Team', 'Appointed'])
# Fillna with end date of 2022-2023 season
head_coach_bis['EndDate'] = head_coach_bis['EndDate'].fillna('2024-01-14')
# Check if the next appointment is overlapping with the current one
head_coach_bis['OverlapDuration'] = head_coach_bis.groupby('Team')['Appointed'].shift(-1) - head_coach_bis['EndDate']
head_coach_bis['Overlap'] = head_coach_bis['OverlapDuration'].dt.days < 0

# Show team with overlapping appointments
overlapping = head_coach_bis[head_coach_bis['Overlap']]

There is {eval}`overlapping.shape[0]` inconsistent record of head coach in {eval}`' ,'.join(overlapping['Team'].unique())` teams.

In [11]:
head_coach_bis.sort_values('OverlapDuration').head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,OverlapDuration,Overlap
3392,Stade Reims,Ligue 1,France,David Guion,2017-05-22,2021-05-25,1464,157,67,46,44,-1059 days,True
3230,Stade Rennais FC,Ligue 1,France,Rolland Courbis,2016-01-20,2016-09-11,235,17,6,3,8,-72 days,True
3313,LOSC Lille,Ligue 1,France,René Girard,2013-07-01,2015-06-30,729,95,42,24,29,-35 days,True
2002,Torino FC,Serie A,Italy,Gian Piero Ventura,2011-06-06,2016-06-25,1846,217,85,64,68,-31 days,True
190,West Ham United,Premier League,England,Sam Allardyce,2011-07-01,2015-06-30,1460,181,69,44,68,-21 days,True


In [12]:
head_coach_bis[head_coach_bis['Team'].isin(overlapping['Team'])][['Team', 'Appointed', 'EndDate', 'Overlap', 'OverlapDuration']].sort_values(['Team', 'Appointed']).head(10)

,Team,Appointed,EndDate,Overlap,OverlapDuration
2247,FC Empoli,2012-06-25,2015-06-04,False,11 days
2246,FC Empoli,2015-06-15,2016-06-30,False,1 days
2245,FC Empoli,2016-07-01,2017-06-30,False,1 days
2244,FC Empoli,2017-07-01,2017-12-17,False,0 days
2243,FC Empoli,2017-12-17,2018-11-05,False,1 days
2242,FC Empoli,2018-11-06,2019-03-13,False,0 days
2241,FC Empoli,2019-03-13,2019-06-30,True,-12 days
2240,FC Empoli,2019-06-18,2019-11-12,False,2 days
2239,FC Empoli,2019-11-14,2020-01-26,False,0 days
2238,FC Empoli,2020-01-26,2020-08-11,False,8 days


In [13]:
#| label: hc_inconsistency
head_coach_bis[head_coach_bis['Team'] == 'Stade Reims'][['Team', 'Appointed', 'EndDate', 'Overlap']].sort_values(['Appointed'])

,Team,Appointed,EndDate,Overlap
3395,Stade Reims,2014-07-01,2015-04-07,False
3394,Stade Reims,2015-04-07,2016-04-23,False
3393,Stade Reims,2016-07-01,2017-05-22,False
3392,Stade Reims,2017-05-22,2021-05-25,True
3391,Stade Reims,2018-07-01,2019-03-30,False
3390,Stade Reims,2021-06-23,2022-10-13,False
3389,Stade Reims,2022-10-13,2024-01-14,False


In [14]:
# Check records that are overlapping
head_coach[~head_coach.index.isin(head_coach_bis[head_coach_bis['OverlapDuration'].dt.days <= -20].index)]
# Drop overlapping records with a duration of more than 20 days
head_coach = head_coach[head_coach.index.isin(head_coach_bis[head_coach_bis['OverlapDuration'].dt.days > -20].index)]

Next we would like to add to each head coach record the number of appointment he is completing.

In [15]:
# Using cronological information about appointment
# Add a column to head coach records that tells us about how many appointment head coach has done

head_coach = head_coach.sort_values(['HeadCoach', 'Appointed'])
head_coach['AppointmentNumber'] = head_coach.groupby('HeadCoach').cumcount() + 1
head_coach[head_coach['HeadCoach'].isin(head_coach[head_coach['AppointmentNumber'] > 3]['HeadCoach'])].sort_values('HeadCoach').head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
2243,FC Empoli,Serie A,Italy,Aurelio Andreazzoli,2017-12-17,2018-11-05,323,35,17,10,8,1
2241,FC Empoli,Serie A,Italy,Aurelio Andreazzoli,2019-03-13,2019-06-30,109,11,5,1,5,2
1773,Genoa CFC,Serie A,Italy,Aurelio Andreazzoli,2019-07-01,2019-10-22,113,9,2,2,5,3
2236,FC Empoli,Serie A,Italy,Aurelio Andreazzoli,2021-06-21,2022-06-01,345,41,12,11,18,4
415,Real Madrid,LaLiga,Spain,Carlo Ancelotti,2013-07-01,2015-05-25,693,119,89,14,16,1


In [16]:
# Check if total_matches = wins + draws + losses
head_coach[head_coach['Matches'] != head_coach['Wins'] + head_coach['Draws'] + head_coach['Losses']].shape[0]

0

In [17]:
# Display Head Coach with lowest number of match
display(head_coach.sort_values('Matches').head(20))
# Remove head coach records with less than 5 matches
# We consider them to be not relevant as it a short stay of a head coach indicate either a temporary replacement or a very bad performance
head_coach = head_coach[head_coach['Matches'] > 5]

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
1696,SS Lazio,Serie A,Italy,Marcelo Bielsa,2016-07-06,2016-07-08,2,0,0,0,0,2
3391,Stade Reims,Ligue 1,France,Sébastien Desmazeau,2018-07-01,2019-03-30,272,0,0,0,0,1
2669,TSG 1899 Hoffenheim,Bundesliga,Germany,Marcel Rapp,2020-06-25,2020-06-30,5,1,1,0,0,1
1920,Udinese Calcio,Serie A,Italy,Igor Tudor,2018-04-24,2018-06-30,67,4,2,1,1,1
2732,VfB Stuttgart,Bundesliga,Germany,Nico Willig,2019-05-06,2019-06-30,55,4,1,3,0,1
2303,Hellas Verona,Serie A,Italy,Eusebio Di Francesco,2021-06-07,2021-09-14,99,4,1,0,3,3
2736,VfB Stuttgart,Bundesliga,Germany,Jos Luhukay,2016-07-01,2016-09-15,76,5,3,0,2,1
243,Crystal Palace,Premier League,England,Frank de Boer,2017-07-01,2017-09-11,72,5,1,0,4,2
932,Granada CF,LaLiga,Spain,Miguel Ángel Portugal,2018-05-01,2018-06-30,60,5,2,0,3,1
3360,Montpellier HSC,Ligue 1,France,Pascal Baills,2015-12-27,2016-01-26,30,5,1,0,4,1


#### Joining head coach with match results

Nous ajoutons aux données de match l'ancienneté du coach au sein de l'équipe lorsque ce match a été joué. Cela nous permettra de voir si l'ancienneté de l'entraîneur au sein d'une équipe a un impact sur sa performance.

Cependant, le nom des équipes dans les deux jeux de données sont différents. Nous devons corriger cela avant de pouvoir joindre les deux dataframes.

In [18]:
# Compute number of team that are in head_coach but not in match_results
coach_teams = set(head_coach['Team'])
match_teams = set(match_results['Team'])

coach_team_not_in_match = coach_teams - match_teams
match_team_not_in_coach = match_teams - coach_teams

len(coach_team_not_in_match), len(match_team_not_in_coach)

(58, 153)

Au total, les résultats de matchs contiennent {eval}`len(match_team)` équipes et les mandats de coachs contiennent {eval}`len(coach_team) équipes. Cependant, le nom de certaines équipes est différent entre les deux jeux de données. Par exemple, « Liverpool » dans les résultats de match devient « Liverpool FC » dans les mandats des entraîneurs.

Il existe {eval}`len(coach_team_not_in_match)` références à des équipes dans les mandats de coachs qui n'ont pas de correspondance dans les résultats de match.

Nous utiliserons la distance de Levenshtein afin de faire correspondre les équipes des mandats d'entraîneurs aux équipes des résultats de matchs. Nous restreignons la recherche d'une équipe correspondante à la liste des équipes du pays correspondant afin de limiter le nombre de correspondances possibles.
L'ajustement du score de Levenshtein permet de rapidement vérifier la validité des correspondances de moins bonne certitudes.

In [19]:
# Group teams by country
coach_teams_by_country = head_coach.groupby('Country')['Team'].unique()
match_teams_by_country = match_results.groupby('Country')['Team'].unique()

coach_teams_by_country, match_teams_by_country

(Country
 England    [Newcastle United, Crystal Palace, Chelsea FC,...
 France     [OGC Nice, FC Toulouse, Olympique Marseille, F...
 Germany    [1.FC Köln, 1.FSV Mainz 05, Eintracht Frankfur...
 Italy      [Frosinone Calcio, FC Empoli, Genoa CFC, Hella...
 Spain      [Granada CF, Valencia CF, Rayo Vallecano, Celt...
 Name: Team, dtype: object,
 Country
 England    [Manchester Utd, Stoke City, Leicester City, Q...
 France     [Reims, Nice, Guingamp, Nantes, Evian, Lille, ...
 Germany    [Bayern Munich, Hoffenheim, Hannover 96, Köln,...
 Italy      [Chievo, Roma, Atalanta, Milan, Genoa, Palermo...
 Spain      [Málaga, Granada, Sevilla, Almería, Eibar, Cel...
 Name: Team, dtype: object)

In [20]:
from thefuzz import process

team_name_mapping = {}

# For each country
for country in coach_teams_by_country.index:
    # Get teams for this country
    coach_teams = coach_teams_by_country[country]
    match_teams = match_teams_by_country.get(country, [])

    # For each team in coach_teams
    for coach_team in coach_teams:
        # Find the best match in match_teams
        matching_scores = process.extract(coach_team, match_teams, limit=1)

        if len(matching_scores) != 0 and matching_scores[0][1] >= 60:
            team_name_mapping[coach_team] = matching_scores[0][0]
        else:
            team_name_mapping[coach_team] = None
            print(f"No match found for {coach_team} among {match_teams} in {country}")

In [21]:
#| label: team_match_table

name_match = pd.DataFrame(team_name_mapping.items(), columns=['Team in head coach records', 'Team in match results'])
display(HTML(name_match.head().to_html(index=False)))

Team in head coach records,Team in match results
Newcastle United,Newcastle Utd
Crystal Palace,Crystal Palace
Chelsea FC,Chelsea
Tottenham Hotspur,Tottenham
Arsenal FC,Arsenal


In [22]:
# Map head_coach['team'] with name_match
head_coach['Team'] = head_coach['Team'].map(team_name_mapping)
head_coach.head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
940,Granada,LaLiga,Spain,Abel Resino,2015-01-19,2015-05-01,102,15,2,5,8,1
2906,Köln,Bundesliga,Germany,Achim Beierlorzer,2019-07-01,2019-11-09,131,13,3,1,9,1
2687,Mainz 05,Bundesliga,Germany,Achim Beierlorzer,2019-11-18,2020-09-28,315,26,9,4,13,2
2790,Eint Frankfurt,Bundesliga,Germany,Adi Hütter,2018-07-01,2021-06-30,1095,141,67,31,43,1
2633,M'Gladbach,Bundesliga,Germany,Adi Hütter,2021-07-01,2022-06-30,364,37,14,9,14,2


Nous pouvons maintenant ajouter le nombres de jours au poste d’entraîneur avec les résultats de match.

In [23]:
#| label: final_match_results

# Merge head_coach with match_results
match = match_results.merge(head_coach[['Team', 'HeadCoach', 'Appointed', 'EndDate']], on=['Team'], how='left')
# Put aside team that don't have a head coach
no_headcoach = match[match['HeadCoach'].isna()]
match = match[~match['HeadCoach'].isna()]
# Filter match_results_bis to keep only head coach that were appointed before the match and with no end date or end date after the match
match = match[
    (match['Date'] >= match['Appointed']) &
    ((match['Date'] <= match['EndDate']) | match['EndDate'].isna())]
# Join back the team that don't have a head coach
match = pd.concat([match, no_headcoach], ignore_index=True)
# Compute daysInPost
match['DaysInPost'] = (match['Date'] - match['Appointed']).dt.days
match = match.drop(columns=['Appointed', 'EndDate'])
match.head()

,League,Country,Date,Team,Goals,Result,isHome,HeadCoach,DaysInPost
0,Premier League,England,2014-08-16,Manchester Utd,1.0,loss,True,Louis van Gaal,33.0
1,Premier League,England,2014-08-16,Arsenal,2.0,win,True,Arsène Wenger,6528.0
2,Premier League,England,2014-08-17,Liverpool,2.0,win,True,Brendan Rodgers,777.0
3,Premier League,England,2014-08-17,Newcastle Utd,0.0,loss,True,Alan Pardew,1347.0
4,Premier League,England,2014-11-01,Newcastle Utd,1.0,win,True,Alan Pardew,1423.0


### Saving preprocessed data

In [24]:
# Save match_results
match.to_csv('data/match_results.csv', index=False)
head_coach.to_csv('data/head_coach.csv', index=False)